In [1]:
import numpy as np
import pandas as pd

In [5]:
ratings = pd.read_csv('ratings.dat', sep='::', encoding='latin-1', engine='python')
users = pd.read_csv('users.dat', sep='::', encoding='latin-1', engine='python')
movies = pd.read_csv('movies.dat',sep='::',encoding='latin-1', engine='python')

In [6]:
movies['Genres'] = movies['Genres'].str.split('|')
unique_genres = set(genre for genres_list in movies['Genres'] for genre in genres_list)
g_1hot = pd.DataFrame(0, index = movies['MovieID'], columns= list(unique_genres))
for i, gs in enumerate(movies['Genres']):
    for g in gs:
        g_1hot.loc[i, g] = 1

In [14]:
g_1hot = g_1hot.fillna(0)

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer


In [16]:
tfidf_tran = TfidfTransformer()
tfidf = tfidf_tran.fit_transform(g_1hot.to_numpy())
tfidf_matrix = tfidf.toarray()
tfidf_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.81877379, 0.        , 0.        , ..., 0.57411626, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.75520046, 0.        ,
        0.        ],
       ...,
       [0.85422932, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.77610736, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.81877379, 0.        , 0.        , ..., 0.57411626, 0.        ,
        0.        ]])

In [17]:
data_dim = tfidf.shape[1]
movies_dim = tfidf.shape[0]
users_dim = users.shape[0]
W = np.zeros((data_dim, users_dim))
b = np.zeros((1,users_dim))


In [18]:
ratings.drop(columns='Timestamp', inplace=True)
ratings

,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [19]:
data = ratings.to_numpy()
data

array([[   1, 1193,    5],
       [   1,  661,    3],
       [   1,  914,    3],
       ...,
       [6040,  562,    5],
       [6040, 1096,    4],
       [6040, 1097,    4]])

In [20]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

In [21]:
#x_train, x_test, y_train, y_test = train_test_split(ratings.iloc[:,[0,1]], ratings.iloc[:,[2]],shuffle=True, test_size=0.18, random_state=42,stratify=data[:,0] )
x_train, x_test, y_train, y_test = train_test_split(ratings.iloc[:,[0,1]], ratings.iloc[:,[2]],shuffle=True, test_size=0.18, random_state=42,stratify=data[:,0] )


In [22]:
xdata_train = x_train.to_numpy()
ydata_train = y_train.to_numpy()
xdata_test = x_test.to_numpy()
ydata_test = y_test.to_numpy()



In [46]:
itemM = xdata_train[xdata_train[:,0] == 1,1]
itemMM = ydata_train[xdata_train[:,0] == 1]

In [23]:
def get_item_rating(x_trainn, y_trainn, user_id):
    item_index = x_trainn[x_trainn[:,0] == (user_id+1), 1]
    rating = y_trainn[x_trainn[:, 0] == (user_id+1)]
    return (item_index-1, rating)

In [24]:
import sklearn.linear_model

In [71]:
b.shape


(1, 6040)

In [25]:
for j in range(users_dim):
    index, ratingg = get_item_rating(xdata_train, ydata_train, j)
    model = sklearn.linear_model.LinearRegression(fit_intercept=True)
    X = tfidf_matrix[index, :]
    model.fit(X, ratingg)
    W[:, j] = model.coef_
    b[0, j] = model.intercept_

/tmp/ipykernel_22006/2682095953.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b[0, j] = model.intercept_
/tmp/ipykernel_22006/2682095953.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b[0, j] = model.intercept_
/tmp/ipykernel_22006/2682095953.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b[0, j] = model.intercept_
/tmp/ipykernel_22006/2682095953.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error

In [28]:
y = tfidf_matrix.dot(W) + b
y

array([[5.        , 4.12345543, 3.94647815, ..., 4.77384548, 2.81691015,
        2.60357046],
       [4.15057205, 3.95339086, 3.65576348, ..., 3.30090273, 3.77910141,
        3.46517105],
       [4.25282733, 3.71381999, 4.49564422, ..., 3.26344964, 4.11374139,
        3.69715179],
       ...,
       [4.30629786, 3.52940498, 3.15649964, ..., 3.54662682, 3.72612003,
        3.48737412],
       [3.78087351, 3.92079993, 3.3905245 , ..., 4.6663062 , 3.40492842,
        3.6102609 ],
       [4.15057205, 3.95339086, 3.65576348, ..., 3.30090273, 3.77910141,
        3.46517105]])

array([[4],
       [4],
       [2],
       [5],
       [4],
       [3],
       [4],
       [5],
       [5],
       [1],
       [4],
       [5],
       [5],
       [5],
       [5],
       [4],
       [5]])

In [35]:
y[3, test11].shape

(17,)

In [ ]:
def RMSE(ypre, x, y, W, b):
    sum, count = 0, 0 
    for n in range(users_dim):
        index, rating = get_item_rating(x, y, n)
        rating_pre = ypre[index, n]
        loss = rating_pre - rating
        sum += loss*loss.sum(axis=0)
        count += rating.shape[0]
    return np.sqrt(sum/count)

In [32]:
Wlasso = np.zeros((data_dim, users_dim))
blasso = np.zeros((1,users_dim))
for j in range(users_dim):
    index, ratingg = get_item_rating(xdata_train, ydata_train, j)
    model = sklearn.linear_model.Lasso(fit_intercept=True)
    X = tfidf_matrix[index, :]
    model.fit(X, ratingg)
    Wlasso[:, j] = model.coef_
    blasso[0, j] = model.intercept_

/tmp/ipykernel_22006/4085774175.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  blasso[0, j] = model.intercept_
/tmp/ipykernel_22006/4085774175.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  blasso[0, j] = model.intercept_
/tmp/ipykernel_22006/4085774175.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  blasso[0, j] = model.intercept_
/tmp/ipykernel_22006/4085774175.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated,

In [1]:
Wridge = np.zeros((data_dim, users_dim))
bridge = np.zeros((1,users_dim))
for j in range(users_dim):
    index, ratingg = get_item_rating(xdata_train, ydata_train, j)
    model = sklearn.linear_model.Ridge(alpha=0.1, fit_intercept=True)
    X = tfidf_matrix[index, :]
    model.fit(X, ratingg)
    Wridge[:, j] = model.coef_
    bridge[0, j] = model.intercept_

NameError: name 'np' is not defined

In [ ]:
Welastic = np.zeros((data_dim, users_dim))
belastic = np.zeros((1,users_dim))
for j in range(users_dim):
    index, ratingg = get_item_rating(xdata_train, ydata_train, j)
    model = sklearn.linear_model.ElasticNet(alpha=0.1,fit_intercept=True)
    X = tfidf_matrix[index, :]
    model.fit(X, ratingg)
    Welastic[:, j] = model.coef_
    belastic[0, j] = model.intercept_